_My notebook on_
# Python for Data Analysis - Wes McKinney
## Chapter 10 - Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## Part 3 - Apply: General split-apply-combine

In [2]:
tips = pd.read_csv('examples/tips.csv')

In [3]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [4]:
def top(df, n=5, column='tip_pct'):
    """selects the rows with the largest values in a given column"""
    return df.sort_values(by=column)[-n:]

In [5]:
# six top tips by tip_pct
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [6]:
# group tips by smoker, then apply the top() function above
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [7]:
# group tips by smoker and day, then apply top() to get a single item for each on total_bill
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [8]:
# describe tip_pct on tips after grouping on smoker
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [9]:
# describe() here, is an unstacked shortcut for
tips.groupby('smoker')['tip_pct'].apply(lambda x: x.describe()).unstack()

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [10]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Suppressing the Group Keys

In [11]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Quantile and Bucket Analysis

In [12]:
frame = pd.DataFrame({
    'data1': np.random.randn(1000),
    'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0     (0.00541, 1.858]
1    (-1.847, 0.00541]
2    (-1.847, 0.00541]
3     (0.00541, 1.858]
4    (-1.847, 0.00541]
5     (0.00541, 1.858]
6     (0.00541, 1.858]
7     (0.00541, 1.858]
8     (0.00541, 1.858]
9    (-1.847, 0.00541]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.707, -1.847] < (-1.847, 0.00541] < (0.00541, 1.858] < (1.858, 3.71]]

In [13]:
# The Categorical object returned by cut can be passed directly to groupby
def get_stats(group):
    return {
        'min': group.min(),
        'max': group.max(),
        'count': group.count(),
        'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.707, -1.847]",47.0,2.580689,0.051295,-1.913732
"(-1.847, 0.00541]",463.0,2.809198,-0.022237,-2.777351
"(0.00541, 1.858]",459.0,2.819306,-0.047432,-2.693707
"(1.858, 3.71]",31.0,3.530097,-0.034170,-2.110104


In [14]:
# equal-size buckets based on sample quantiles: qcut
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.809198,-0.019643,-2.386650
1,100.0,2.091171,-0.114985,-2.777351
2,100.0,2.180903,0.117041,-2.414401
3,100.0,2.300349,-0.008096,-2.399095
4,100.0,2.009443,-0.010813,-2.598458
5,100.0,2.524541,-0.067866,-2.693707
6,100.0,2.819306,0.123209,-2.599414
7,100.0,2.716371,-0.115958,-2.400605
8,100.0,1.999811,-0.115903,-2.529598


Example: Filling Missing Values with Group-Specific Values

In [15]:
# a series with NaN in it
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
print(s)

# replacing NaN with the mean
s = s.fillna(s.mean())
print(s)

0         NaN
1   -1.554576
2         NaN
3    0.843738
4         NaN
5    0.550815
dtype: float64
0   -0.053341
1   -1.554576
2   -0.053341
3    0.843738
4   -0.053341
5    0.550815
dtype: float64


In [16]:
# fill value could vary by group
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
print('--')
print(data)
print('--')
print(data.groupby(group_key).mean())
print('--')

# fill the NA values using the group means
print(data.groupby(group_key).apply(lambda g: g.fillna(g.mean())))

# fill with predefined values
fill_values = {'East': 0.5, 'West': -1}
print(data.groupby(group_key).apply(lambda g: g.fillna(fill_values[g.name])))

--
Ohio          1.163909
New York      0.980455
Vermont            NaN
Florida       0.403404
Oregon        0.846198
Nevada             NaN
California   -1.583178
Idaho              NaN
dtype: float64
--
East    0.849256
West   -0.368490
dtype: float64
--
Ohio          1.163909
New York      0.980455
Vermont       0.849256
Florida       0.403404
Oregon        0.846198
Nevada       -0.368490
California   -1.583178
Idaho        -0.368490
dtype: float64
Ohio          1.163909
New York      0.980455
Vermont       0.500000
Florida       0.403404
Oregon        0.846198
Nevada       -1.000000
California   -1.583178
Idaho        -1.000000
dtype: float64


Example: Random Sampling and Permutation

In [17]:
# a deck of English-style playing cards

# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'Q', 'K']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
QH     10
KH     10
dtype: int64

In [18]:
def draw(deck, n=5):
    """sample w/o repetition from a deck"""
    return deck.sample(n)

draw(deck)

10S    10
10D    10
5D      5
KS     10
3D      3
dtype: int64

In [20]:
# two random cards from each suit - last letter is suit
print(deck.groupby(lambda card: card[-1]).apply(draw, n=2))

# same, but w/o showing group keys
print(deck.groupby(lambda card: card[-1], group_keys=False).apply(draw, n=2))

C  7C     7
   AC     1
D  7D     7
   KD    10
H  2H     2
   3H     3
S  4S     4
   6S     6
dtype: int64
4C      4
5C      5
KD     10
10D    10
7H      7
AH      1
10S    10
2S      2
dtype: int64


Example: Group Weighted Average and Correlation

In [21]:
df = pd.DataFrame({
    'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'],
    'data': np.random.randn(8),
    'weights': np.random.rand(8)
})

print(df)
print('--')

# group weighted average by category
print(df.groupby('category').apply(lambda g: np.average(g['data'], weights=g['weights'])))

  category      data   weights
0        a  0.757615  0.388382
1        a  1.387404  0.792786
2        a  1.403939  0.778664
3        a -0.533597  0.194592
4        b -0.194020  0.812082
5        b  1.404767  0.709304
6        b  0.009844  0.002330
7        b -1.575059  0.754895
--
category
a    1.106338
b   -0.153661
dtype: float64


In [22]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [23]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()
close_px[-4:]

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [24]:
# yearly correlations of daily returns (computed from percent changes) with SPX
spx_corr = lambda x: x.corrwith(x['SPX'])

rets = close_px.pct_change().dropna()
print(rets[-4:])

get_year = lambda x: x.year
by_year = rets.groupby(get_year)
print(by_year.apply(spx_corr))

# annual correlation between Apple and Microsoft
print('--')
print(by_year.apply(lambda g: g['AAPL'].corr(g['MSFT'])))

                AAPL      MSFT       XOM       SPX
2011-10-11  0.029526  0.002227 -0.000131  0.000544
2011-10-12  0.004747 -0.001481  0.011669  0.009795
2011-10-13  0.015515  0.008160 -0.010238 -0.002974
2011-10-14  0.033225  0.003311  0.022784  0.017380
          AAPL      MSFT       XOM  SPX
2003  0.541124  0.745174  0.661265  1.0
2004  0.374283  0.588531  0.557742  1.0
2005  0.467540  0.562374  0.631010  1.0
2006  0.428267  0.406126  0.518514  1.0
2007  0.508118  0.658770  0.786264  1.0
2008  0.681434  0.804626  0.828303  1.0
2009  0.707103  0.654902  0.797921  1.0
2010  0.710105  0.730118  0.839057  1.0
2011  0.691931  0.800996  0.859975  1.0
--
2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64


Example: Group-Wise Linear Regression

In [25]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

D:\dev\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [26]:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## Part 4 - Pivot Tables and Cross-Tabulation

In [27]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [28]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [29]:
# same as above, but include partial totals
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [30]:
# counting group sizes
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [31]:
# replacing NaN with given value
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

Cross-Tabulations: Crosstab

In [32]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244